In [ ]:
!pip install pyserini
!pip install jsonlines
!apt-get install libomp-dev
!pip install faiss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unzip /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/wikipedia_20220620_cleaned.zip

Archive:  /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data/wikipedia_20220620_cleaned.zip
replace wikipedia_20220620_all_titles.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikipedia_20220620_cleaned.jsonl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import json
import os
# with open('wikipedia_20220620_cleaned.jsonl', 'r') as f:
#     wiki = json.load(f)

In [ ]:
f = open('wikipedia_20220620_cleaned.jsonl', 'r')
samples = []

# for i in range(3):

while True:
    raw = f.readline()
    if not raw:
        break
    raw = eval(raw)
    processed = dict()
    processed['id'] = raw['id']
    processed['contents'] = raw['text']
    processed['contents'] = processed['contents'].replace('"', '\\"')
    processed['title'] = raw['title']
    # samples.append(str(processed).replace('\'', '"'))
    samples.append(processed)

    # break

In [ ]:
len(samples)

1273469

In [ ]:
os.makedirs('lookup_db', exist_ok=True)
os.makedirs('sample_input', exist_ok=True)

In [ ]:
import jsonlines
with jsonlines.open('sample_input/samples.jsonl', 'w') as writer:
    writer.write_all(samples)

In [ ]:
# with open('sample_input/samples.jsonl', 'w') as f:
#     for entry in samples:
#         f.write(entry)
#         # json.dump(entry, f)
#         f.write('\n')

In [ ]:
!python -m pyserini.index.lucene --input sample_input  --index lookup_db --fields title --collection JsonCollection --language vn --generator DefaultLuceneDocumentGenerator --threads 1 --storePositions --storeDocvectors --storeRaw

2022-11-08 14:24:27,044 INFO  [main] index.IndexCollection (IndexCollection.java:250) - Setting log level to INFO
2022-11-08 14:24:27,048 INFO  [main] index.IndexCollection (IndexCollection.java:253) - Starting indexer...
2022-11-08 14:24:27,048 INFO  [main] index.IndexCollection (IndexCollection.java:254) - ============ Loading Parameters ============
2022-11-08 14:24:27,049 INFO  [main] index.IndexCollection (IndexCollection.java:255) - DocumentCollection path: sample_input
2022-11-08 14:24:27,050 INFO  [main] index.IndexCollection (IndexCollection.java:256) - CollectionClass: JsonCollection
2022-11-08 14:24:27,050 INFO  [main] index.IndexCollection (IndexCollection.java:257) - Generator: DefaultLuceneDocumentGenerator
2022-11-08 14:24:27,051 INFO  [main] index.IndexCollection (IndexCollection.java:258) - Threads: 1
2022-11-08 14:24:27,052 INFO  [main] index.IndexCollection (IndexCollection.java:259) - Language: vn
2022-11-08 14:24:27,052 INFO  [main] index.IndexCollection (IndexColl

In [ ]:
# ! python -m pyserini.index.lucene -h

In [ ]:
!zip -r lookup_db.zip lookup_db

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!cp lookup_db.zip /content/drive/MyDrive/AIChallenges/Zalo22/WikiQA/data

In [ ]:
from pyserini.search.lucene import LuceneSearcher

searcher = LuceneSearcher('lookup_db')
searcher.set_language('vn')
question = 'nhạc sĩ nào là tác giả của bài hát nổi tiếng dáng đứng bến tre'
hits = searcher.search(question)

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

 1 38686 30.89020
 2 83269 29.04660
 3 866447 26.39520
 4 11172128 26.11600
 5 420749 24.93180
 6 412067 24.91950
 7 3708412 24.58610
 8 1677259 24.41760
 9 15275083 24.19250
10 233926 24.13280


In [ ]:
import json

In [ ]:
doc = searcher.doc(hits[0].docid)
json_doc = json.loads(doc.raw())
json_doc['contents']

'Nguyễn Văn Tý\n\nNguyễn Văn Tý (5 tháng 3 năm 1925 - 26 tháng 12 năm 2019) là một nhạc sĩ nổi tiếng của Việt Nam, ông có nhiều đóng góp sáng tác từ dòng nhạc tiền chiến như \\"Dư âm\\" đến những ca khúc nhạc đỏ như \\"Dáng đứng Bến Tre\\", \\"Mẹ yêu con\\", \\"Người đi xây hồ Kẻ Gỗ\\"...\n\n== Tiểu sử và sự nghiệp. ==\nNguyễn Văn Tý sinh ngày 5 tháng 3 năm 1925 tại Vinh, Nghệ An, quê gốc ở Phú Cường, Sóc Sơn, Phúc Yên, nay thuộc Hà Nội. Xuất thân trong một gia đình truyền thống âm nhạc, cha của ông là \\"trùm một phường bát âm của miền quê Vĩnh Phú thạo cả hát văn, hát chèo và hát ả đào\\", sau vào làm thợ máy nhà máy xe lửa Trường Thi ở Nghệ An. Thuở bé, Nguyễn Văn Tý học ở trường Quốc học Vinh và được một giáo viên người Pháp dạy cho những bài hát của Tino Rossi đang thịnh hành thời đó. Trong thời gian tham gia hoạt động hướng đạo, ông được một cha cố người Tây Ban Nha cho vào dàn nhạc nhà thờ hát thánh ca. Ở đó ông được học nhạc lý cơ bản và nhất là nâng cao trình độ hòa thanh, hát